# Encontrando APIs ocultas

Cuando una compañia desarrolla un sitio web muchas veces separa lo que se dice el _frontend_, que es la parte visible del sitio y la cual se ejecuta en tu navegador, del _backend_, la parte del sitio que realiza el computo mas pesado y se ejecuta en servidores/computadoras de la compañia.

Para comunicar el _backend_ con el _frontend_ una forma popular es desarrollar REST APIs, a veces estas son públicas pero a veces están ocultas y las utilizamos sin darnos cuenta cuando interactuamos con un sitio web.

Ejemplo de un sitio web y su API: http://numbersapi.com/

Hoy veremos como encontrarlas y utilizarlas para extraer datos de forma cómoda

## Ejemplo 1: Valor del Bitcoin

Para este primer ejemplo veremos como encontrar una API oculta en https://coinmarketcap.com/es

Luego veremos como utilizarla de 3 maneras:
1. Desde el navegador
2. Con cURL (Un programa de linea de comandos para transferir datos de internet)
3. Con Python

[Herramienta](https://www.epochconverter.com/)

### Método utilizando python

Usaremos el sitio web de [coinmarketcap](https://coinmarketcap.com/es/currencies/bitcoin/) que proporciona datos del Bitcoin en tiempo real.

#### Pasos para encontrar la API oculta y utilizarla con Python

1. Entrar al sitio web:

    - Visita el sitio

2. Abrir las herramientas de desarrollo:

    - Presiona Ctrl+Shift+I para abrir las herramientas de desarrollo en tu navegador.
    - Ve a la solapa Network y selecciona XHR.

3. Recargar el sitio y analizar las peticiones:

    - Recarga la página para capturar todas las solicitudes que realiza el sitio.
    - En la pestaña XHR, revisa las peticiones que se están realizando. Busca aquellas que tienen nombres relevantes o cuyo Response contiene datos que te interesen, como el valor del Bitcoin.

4. Identificar la API y copiar el comando cURL:

    - Una vez que encuentres una solicitud relevante, haz clic derecho sobre ella y selecciona "Copy as cURL(bash)" para copiar el comando cURL que realiza la solicitud.

5. Convertir el comando cURL a código Python:

    - Utiliza [curlconverter.com](https://curlconverter.com/) para convertir el comando cURL a código Python. Pega el cURL copiado y el sitio te generará un código en Python utilizando la librería requests.

6. Escribir el código en Python:

    - A partir del código generado, puedes realizar la solicitud a la API y extraer los datos que necesites.

In [ ]:
import requests

headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'es-ES,es;q=0.8',
    'cache-control': 'no-cache',
    'origin': 'https://coinmarketcap.com',
    'platform': 'web',
    'priority': 'u=1, i',
    'referer': 'https://coinmarketcap.com/',
    'sec-ch-ua': '"Chromium";v="128", "Not;A=Brand";v="24", "Brave";v="128"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'sec-gpc': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
    'x-request-id': 'ae6571a8c35944b8863462441f81f1c9',
}

params = {
    'id': '1',
    'range': '1D',
}

response = requests.get('https://api.coinmarketcap.com/data-api/v3/cryptocurrency/detail/chart', params=params, headers=headers)
response.json()

## Mejorando el código

Analizando la API se puede ver que tiene al menos un parámetro modificables, el periodo.

Podemos poner todo dentro de una función que extraiga los precios del Bitcoin para un periodo determinado. De esta forma será más fácil de usar y no necesitemos repetir el código.

In [ ]:
import requests

headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'es-ES,es;q=0.8',
    'cache-control': 'no-cache',
    'origin': 'https://coinmarketcap.com',
    'platform': 'web',
    'priority': 'u=1, i',
    'referer': 'https://coinmarketcap.com/',
    'sec-ch-ua': '"Chromium";v="128", "Not;A=Brand";v="24", "Brave";v="128"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'sec-gpc': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
    'x-request-id': 'ae6571a8c35944b8863462441f81f1c9',
}

params = {
    'id': '1',  # ID de la criptomoneda (1 es Bitcoin)
    'range': '7D',  # Rango de tiempo para los últimos 7 días
}

response = requests.get('https://api.coinmarketcap.com/data-api/v3/cryptocurrency/detail/chart', params=params, headers=headers)

if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print("Error:", response.status_code)


In [ ]:
import pandas as pd

points = data['data']['points']

# Convertir los datos en un DataFrame
df = pd.DataFrame.from_dict(points, orient='index')

# Separar los valores de 'v' en columnas
df[['price', 'volume_24h', 'market_cap', 'c4', 'c5']] = pd.DataFrame(df['v'].tolist(), index=df.index)

# Eliminar las columnas innecesarias ('v' y 'c')
df = df.drop(columns=['v', 'c4', 'c5', 'c'])

# Convertir la columna index (timestamps) a un formato de fecha legible
df.index = pd.to_datetime(pd.to_numeric(df.index), unit='s')

# Mostrar el DataFrame
df

In [ ]:
df['market_cap']/df['price']


In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

# Graficar el Precio
fig.add_trace(go.Scatter(x=df.index, y=df['price'], mode='lines', name='Price (USD)', yaxis='y1'))

# Graficar el Volumen de 24h
fig.add_trace(go.Scatter(x=df.index, y=df['volume_24h'], mode='lines', name='Volume 24h (USD)', yaxis='y2'))

# Configurar los ejes con escala logarítmica
fig.update_layout(
    title='Evolución de Precio, Volumen de 24h y Capitalización de Mercado (Escala Logarítmica)',
    xaxis_title='Fecha',
    yaxis=dict(
        title='Price (USD)',
        type='log',  # Escala logarítmica
        titlefont=dict(color='blue'),
        tickfont=dict(color='blue'),
        side='left'
    ),
    yaxis2=dict(
        title='Volume 24h (USD)',
        type='log',  # Escala logarítmica
        titlefont=dict(color='orange'),
        tickfont=dict(color='orange'),
        overlaying='y',
        side='right'
    ),
)

# Mostrar la figura
fig.show()


## Ejercicio Meteorológica

Conseguir los datos de la meteorología de Madrid: https://www.bbc.com/weather/3117735

In [1]:
import requests

headers = {
    'sec-ch-ua-platform': '"Windows"',
    'Referer': 'https://www.bbc.com/',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
}

response = requests.get('https://weather-broker-cdn.api.bbci.co.uk/en/forecast/aggregated/3117735', headers=headers)

In [64]:
data_weather = response.json()

In [36]:
temp_per_hour = data_weather['forecasts'][0]['detailed']['reports'][0]['temperatureC']

In [79]:
forecasts = data_weather['forecasts']
records = []

for forecast in forecasts:
    summary = forecast['summary']
    report = summary['report']
    
    record = {
        'localDate': summary['issueDate']
        'Temperature': report[]
    }

{'issueDate': '2025-01-10T22:42:00+01:00', 'lastUpdated': '2025-01-11T10:01:25.277+01:00', 'report': {'enhancedWeatherDescription': 'Sunny intervals and a gentle breeze', 'gustSpeedKph': 23, 'gustSpeedMph': 14, 'localDate': '2025-01-11', 'lowermaxTemperatureC': None, 'lowermaxTemperatureF': None, 'lowerminTemperatureC': None, 'lowerminTemperatureF': None, 'maxTempC': 18, 'maxTempF': 64, 'minTempC': 5, 'minTempF': 40, 'mostLikelyHighTemperatureC': 19, 'mostLikelyHighTemperatureF': 66, 'mostLikelyLowTemperatureC': 1, 'mostLikelyLowTemperatureF': 34, 'pollenIndex': None, 'pollenIndexBand': None, 'pollenIndexIconText': None, 'pollenIndexText': None, 'pollutionIndex': None, 'pollutionIndexBand': None, 'pollutionIndexIconText': None, 'pollutionIndexText': None, 'precipitationProbabilityInPercent': 0, 'precipitationProbabilityText': 'Precipitation is not expected', 'sunrise': '08:38', 'sunset': '18:08', 'uppermaxTemperatureC': None, 'uppermaxTemperatureF': None, 'upperminTemperatureC': None, 

In [74]:
records

[]

#### Crear el DataFrame

#### Trazar un gráfico

In [37]:
temp_per_hour

12